# DATAWAREHOUSING

In [0]:
%sql
CREATE DATABASE IF NOT EXISTS company.orders_dwh

### Stagging Layer

In [0]:
%sql
-- Initial Load
CREATE OR REPLACE TABLE company.orders_dwh.stg_sales AS
SELECT * FROM company.sales.orders

num_affected_rows,num_inserted_rows


### Transformation

In [0]:
%sql
CREATE OR REPLACE VIEW company.orders_dwh.vw_trans_sales
AS
SELECT 
    CAST(OrderID AS INT) AS order_id,
    CAST(OrderDate AS DATE) AS order_date,
    CAST(CustomerID AS INT) AS customer_id,
    CAST(CustomerName AS VARCHAR(100)) AS customer_name,
    CAST(CustomerEmail AS VARCHAR(100)) AS customer_email,
    CAST(ProductID AS INT) AS product_id,
    CAST(ProductName AS VARCHAR(100)) AS product_name,
    CAST(ProductCategory AS VARCHAR(50)) AS product_category,
    CAST(RegionID AS INT) AS region_id,
    CAST(RegionName AS VARCHAR(50)) AS region_name,
    CAST(Country AS VARCHAR(50)) AS country,
    CAST(Quantity AS INT) AS quantity,
    CAST(UnitPrice AS DECIMAL(10,2)) AS unit_price,
    CAST(TotalAmount AS DECIMAL(10,2)) AS total_amount
FROM company.orders_dwh.stg_sales
WHERE Quantity IS NOT NULL;

### Core Layer

#### Dim Customers

In [0]:
%sql
CREATE OR REPLACE TABLE company.orders_dwh.dim_customers
(
  dim_customers_key INT,
  customer_id INT,
  customer_name STRING,
  customer_email STRING
)

In [0]:
%sql
CREATE OR REPLACE VIEW company.orders_dwh.vw_dim_customers AS
SELECT
row_number() over(ORDER BY customer_id ASC) as dim_customers_key,
customers.*
FROM
(
  SELECT DISTINCT
    customer_id,
    customer_name,
    customer_email
  FROM company.orders_dwh.vw_trans_sales
) as customers


In [0]:
%sql
INSERT INTO company.orders_dwh.dim_customers
SELECT * FROM company.orders_dwh.vw_dim_customers

num_affected_rows,num_inserted_rows
7,7


#### Dim Products

In [0]:
%sql
CREATE OR REPLACE TABLE company.orders_dwh.dim_products
(
  dim_products_key INT,
  product_id INT,
  product_name STRING,
  product_category STRING
)

In [0]:
%sql
CREATE OR REPLACE VIEW company.orders_dwh.vw_dim_products
AS 
SELECT row_number() over(ORDER BY products.product_id) as dim_products_key,
products.*
FROM 
(
  SELECT DISTINCT
    product_id,
    product_name,
    product_category
  FROM company.orders_dwh.vw_trans_sales
) AS products

In [0]:
%sql
INSERT INTO company.orders_dwh.dim_products
SELECT * FROM company.orders_dwh.vw_dim_products

num_affected_rows,num_inserted_rows
9,9


#### Dim Regions

In [0]:
%sql
CREATE OR REPLACE TABLE company.orders_dwh.dim_regions
(
  dim_regions_key INT,
  region_id INT,
  region_name STRING,
  country STRING
)

In [0]:
%sql
CREATE OR REPLACE VIEW company.orders_dwh.vw_dim_regions
AS 
SELECT row_number() over(ORDER BY regions.region_id) as dim_regions_key,
regions.*
FROM 
(
  SELECT DISTINCT
    region_id,
    region_name,
    country
  FROM company.orders_dwh.vw_trans_sales
) AS regions

In [0]:
%sql
INSERT INTO company.orders_dwh.dim_regions
SELECT * FROM company.orders_dwh.vw_dim_regions

num_affected_rows,num_inserted_rows
8,8


#### Dim Dates

In [0]:
%sql
CREATE OR REPLACE TABLE company.orders_dwh.dim_dates
(
  dim_dates_key INT,
  order_date Date
)

In [0]:
%sql
CREATE OR REPLACE VIEW company.orders_dwh.vw_dim_dates
AS 
SELECT row_number() over(ORDER BY dates.order_date) as dim_dates_key,
dates.*
FROM 
(
  SELECT DISTINCT
    order_date
  FROM company.orders_dwh.vw_trans_sales
) AS dates

In [0]:
%sql
INSERT INTO company.orders_dwh.dim_dates
SELECT * FROM company.orders_dwh.vw_dim_dates

num_affected_rows,num_inserted_rows
10,10


### FACT TABLE

In [0]:
%sql
CREATE TABLE company.orders_dwh.fact_sales
(
  order_id INT,
  quantity DECIMAL,
  unit_price DECIMAL,
  total_amount DECIMAL,
  dim_products_key INT,
  dim_customers_key INT,
  dim_regions_key INT,
  dim_dates_key INT
)

In [0]:
%sql
INSERT INTO company.orders_dwh.fact_sales
SELECT 
  F.order_id,
  F.quantity,
  F.unit_price,
  F.total_amount,
  DC.dim_customers_key,
  DP.dim_products_key,
  DR.dim_regions_key,
  DD.dim_dates_key
FROM  
  company.orders_dwh.vw_trans_sales F 
LEFT JOIN 
  company.orders_dwh.dim_customers DC 
  ON F.customer_id = DC.customer_id
LEFT JOIN 
  company.orders_dwh.dim_products DP 
  ON F.product_id = DP.product_id
LEFT JOIN 
  company.orders_dwh.dim_regions DR 
  ON DR.country = F.country
LEFT JOIN 
  company.orders_dwh.dim_dates DD 
  ON F.order_date = DD.order_date

num_affected_rows,num_inserted_rows
10,10


In [0]:
%sql
SELECT * FROM company.orders_dwh.fact_sales

order_id,quantity,unit_price,total_amount,dim_products_key,dim_customers_key,dim_regions_key,dim_dates_key
1,2,800,1600,1,1,2,1
2,1,500,500,2,2,4,2
3,3,300,900,3,3,7,3
4,1,150,150,1,4,2,4
5,1,400,400,4,5,3,5
6,2,200,400,2,6,6,6
7,1,800,800,5,1,1,7
8,2,250,500,6,7,5,8
9,3,100,300,7,8,8,9
10,1,50,50,4,9,2,10


### Business Questions

In [0]:
%sql
-- ¿Qué productos se venden más en cada región?
WITH ranked_products AS (
  SELECT
    dr.region_name,
    dp.product_name,
    SUM(fs.quantity) AS total_quantity,
    ROW_NUMBER() OVER (PARTITION BY dr.region_name ORDER BY SUM(fs.quantity) DESC) AS rank
  FROM company.orders_dwh.fact_sales fs
  LEFT JOIN company.orders_dwh.dim_regions dr ON fs.dim_regions_key = dr.dim_regions_key
  LEFT JOIN company.orders_dwh.dim_products dp ON fs.dim_products_key = dp.dim_products_key
  GROUP BY
    dr.region_name,
    dp.product_name
)
SELECT
  rp.region_name,
  rp.product_name,
  rp.total_quantity
FROM ranked_products rp
WHERE rp.rank <= 2

region_name,product_name,total_quantity
Asia,Tablet,3
Asia,Monitor,3
Europe,Smartwatch,2
Europe,Smartphone,1
North America,Laptop,3
North America,Gaming Console,1


In [0]:
%sql
-- ¿Qué clientes generan mayores ingresos?
WITH ranked_customers AS (
    SELECT
        dc.customer_name,
        SUM(fs.total_amount) AS total_income,
        ROW_NUMBER() OVER (ORDER BY SUM(fs.total_amount) DESC) AS rank
    FROM company.orders_dwh.fact_sales fs
    LEFT JOIN company.orders_dwh.dim_customers dc ON fs.dim_customers_key = dc.dim_customers_key
    GROUP BY dc.customer_name
)
SELECT
    rc.customer_name,
    rc.total_income
FROM ranked_customers rc
WHERE rc.rank <= 3


customer_name,total_income
Alice Johnson,2400
Charlie Brown,900
Bob Smith,500


In [0]:
%sql
-- ¿Cómo varían las ventas día a día?
SELECT
    dd.order_date,
    SUM(fs.total_amount) AS total_income
FROM company.orders_dwh.fact_sales fs
LEFT JOIN company.orders_dwh.dim_dates dd ON fs.dim_dates_key = dd.dim_dates_key
GROUP BY dd.order_date
ORDER BY dd.order_date

order_date,total_income
2024-02-01,1600
2024-02-02,500
2024-02-03,900
2024-02-04,150
2024-02-05,400
2024-02-06,400
2024-02-07,800
2024-02-08,500
2024-02-09,300
2024-02-10,50


#### Dimensiones con clave autogenerada

In [0]:
%sql
-- La dimensión genera una key automáticamente
CREATE OR REPLACE TABLE company.orders_dwh.dim_customers_unique
(
  dim_customers_key BIGINT GENERATED ALWAYS AS IDENTITY,
  customer_id INT,
  customer_name STRING,
  customer_email STRING
)

In [0]:
%sql
-- La vista que alimentará la dimensión sólo necesita los campos que se van a actualizar
CREATE OR REPLACE VIEW company.orders_dwh.vw_dim_customers_unique AS
SELECT DISTINCT
  customer_id,
  customer_name,
  customer_email
FROM company.orders_dwh.vw_trans_sales

In [0]:
%sql
-- Proceso actualiza la dimensión si el customer_id existe o inserta en caso contrario
-- Garantizando que el key siempre sea único
MERGE INTO company.orders_dwh.dim_customers_unique dc
USING company.orders_dwh.vw_dim_customers_unique vdc ON dc.customer_id = vdc.customer_id
WHEN MATCHED THEN UPDATE SET
  dc.customer_name = vdc.customer_name,
  dc.customer_email = vdc.customer_email
WHEN NOT MATCHED THEN INSERT
  (customer_id, customer_name, customer_email)
VALUES
  (vdc.customer_id, vdc.customer_name, vdc.customer_email)

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
7,0,0,7


In [0]:
%sql
-- Se observa todos los campos fueron insertados la primera vez
SELECT * FROM company.orders_dwh.dim_customers_unique

dim_customers_key,customer_id,customer_name,customer_email
1,101,Alice Johnson,alice@example.com
2,103,Charlie Brown,charlie@example.com
3,105,Eve Adams,eve@example.com
4,102,Bob Smith,bob@example.com
5,107,Grace White,grace@example.com
6,104,David Lee,david@example.com
7,106,Frank Miller,frank@example.com


In [0]:
%sql
-- Ejemplo aplicando el MERGE a una vista que tiene un customer_id existente y uno nuevo
CREATE VIEW company.orders_dwh.vw_dim_customers_tp AS
SELECT 101 AS customer_id, 'Pepe' as customer_name, 'pepe@utec.com' as customer_email
UNION ALL
SELECT 108 AS customer_id, 'Angel' as customer_name, 'angel@utec.com' as customer_email

In [0]:
%sql
-- Ejecución apuntando a la nueva vista
MERGE INTO company.orders_dwh.dim_customers_unique dc
USING company.orders_dwh.vw_dim_customers_tp vdc ON dc.customer_id = vdc.customer_id
WHEN MATCHED THEN UPDATE SET
  dc.customer_name = vdc.customer_name,
  dc.customer_email = vdc.customer_email
WHEN NOT MATCHED THEN INSERT
  (customer_id, customer_name, customer_email)
VALUES
  (vdc.customer_id, vdc.customer_name, vdc.customer_email)

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
2,1,0,1


In [0]:
%sql
-- Se observa que el customer_id que ya existía fue actualizado y a su vez se generó un nuevo customer_id
SELECT * FROM company.orders_dwh.dim_customers_unique

dim_customers_key,customer_id,customer_name,customer_email
2,103,Charlie Brown,charlie@example.com
3,105,Eve Adams,eve@example.com
4,102,Bob Smith,bob@example.com
5,107,Grace White,grace@example.com
6,104,David Lee,david@example.com
7,106,Frank Miller,frank@example.com
1,101,Pepe,pepe@utec.com
9,108,Angel,angel@utec.com
